# Steam (Environment - Agent)

In [1]:
# Basic import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy.stats import invgamma, gamma
from scipy.stats import t as student
import pdb
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, Dot, Concatenate
from tensorflow.keras.models import Model
tf.config.experimental_run_functions_eagerly(True)
print(tf.__version__)

2.0.0


## Environment

In [2]:
class Environment:
    """ 
    Contextual Multi-Armed Bandit environment
    """
    def __init__(self, nb_films, nb_users, 
                 context_size = 2,
                 displayed_users_embedding_size = 2, #used for the features vector
                 displayed_games_embedding_size = 2, #used for the features vector
                 noise_size = 3,
                 seed=None):     
        
        self._rng = np.random.RandomState(seed)
        #-------------------------------------------------------#
        
        self._nb_games = nb_games
        self._nb_users = nb_users
        self._p = context_size # size of user, size of game
        self._displayed_users_embedding_size = displayed_users_embedding_size
        self._displayed_games_embedding_size = displayed_games_embedding_size
        self._noise_size = noise_size
   
        #-------------------------------------------------------#
    
        self.user_mean = np.ones(self._p)
        self.user_var = np.ones(self._p)
        self.game_mean = np.ones(self._p)
        self.game_var = np.ones(self._p)
        
        #-------------------------------------------------------#
        self.finish = False # flag to know when reset the environment (all games played)
    
    def step(self):
        
        if self.finish:
            print("All games played reset the environment")
            
        """ Choose a game """
        user = self._rng.randint(0, self._nb_users)
        available_games = np.where(self._available_games[user] == 1)[0]
        return user, available_games
    
    def update(self, user, game):
        reward = self._reward_matrix[user, game]
        self._available_games[user, game] = 0
        return reward
    
    def reset(self):
        #self._games = self._rng.uniform(size=(nb_games, context_size))
        #self._users = self._rng.uniform(size=(nb_users, context_size))
        #-------------------------------------------------------#

        self._users = self._rng.normal(loc=self.user_mean,
                                                scale=self.user_var,
                                                size=(self._nb_users, self._p))
        self._games = self._rng.normal(loc=self.game_mean,
                                                scale=self.game_var,
                                                size=(self._nb_games, self._p))
        #-------------------------------------------------------#
        self._reward_matrix = np.zeros((nb_users, nb_games))
        
        for i in range(self._reward_matrix.shape[0]):
            for j in range(self._reward_matrix.shape[1]):
                # reward = np.linalg.norm(self._games[j] - self._users[i], ord=2)
                reward = self._games[j].dot(self._users[i])
                self._reward_matrix[i, j] = reward
                
        self._reward_matrix = ((self._reward_matrix - np.min(self._reward_matrix)) / np.max(self._reward_matrix) * 4).astype(int) + 1
        
        self._available_games = np.ones((nb_users, nb_games))
        self._available_films = np.ones((nb_users, nb_games))
        
        users = deepcopy(self._users)
        
        return users

    def get_feature_vector(self, user, game):
        user_embedding = self._users[user]
        game_embedding = self._games[game]
        
        if self._displayed_users_embedding_size + self._displayed_games_embedding_size > 0:
            variables = np.array([user_embedding[:self._displayed_users_embedding_size],
                                  game_embedding[:self._displayed_games_embedding_size]])

            if self._noise_size > 0:
                noise = self._rng.normal(loc=np.ones(self._noise_size),
                                         scale=np.ones(self._noise_size),
                                         size=self._noise_size)
                
                variables = np.append(variables, noise)
                
            return variables

## Agent

In [3]:
class RandomAgent:
    """ 
    Random agent
    """
    def __init__(self, seed = None):
        self._rng = np.random.RandomState(seed)
    
    def act(self, available_games):
        action = self._rng.choice(available_games)
        return action

## Experiment

In [4]:
# Basic parameter
nb_users = 30 #number of users in the context
nb_games = 10 #number of games in the context
context_size = 2 #number of different film categories = 2

In [5]:
# Creating the environment
env = Environment(nb_games,nb_users,context_size,2020)
env.reset() #reset and initilize the environment

array([[ 1.07139341,  2.6173452 ],
       [ 0.10004728,  0.18273103],
       [ 2.69541486,  0.41168055],
       [ 0.07446554,  2.36395513],
       [ 1.20953751, -1.54659002],
       [ 1.69574011,  1.47757504],
       [ 0.15213019,  2.02126636],
       [ 1.74274769,  0.50308044],
       [ 1.09446731, -1.93724045],
       [ 0.52751627,  1.4322206 ],
       [-0.26069544,  1.35101617],
       [ 1.37336244, -0.88530617],
       [ 1.64315708,  0.41449447],
       [ 0.99366519,  1.19960656],
       [ 0.19233297,  2.98700062],
       [ 1.65652348,  0.68586502],
       [ 1.30794171, -0.17118737],
       [ 1.47940848,  2.01768386],
       [ 0.14115165,  0.89107541],
       [ 1.91547695,  0.33346789],
       [ 0.88550255, -0.37891624],
       [ 2.55044876,  0.72493372],
       [ 0.20285665,  1.84067217],
       [ 0.66825057,  0.73866055],
       [ 2.72554292,  2.97795525],
       [ 2.37740842,  1.13198666],
       [-0.08376984, -0.53701157],
       [ 1.90000765,  0.42722186],
       [ 0.38593846,

In [6]:
# Creating the agent
agent = RandomAgent(2020)

We run the experiment and generate some historical data.

In [7]:
# Running several trials
nb_iteration = 100 #how many trials
rating_matrix = np.zeros((env._nb_users, env._nb_games))
users = list()
games = list()
ratings = list()
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(available_games)
    reward = env.update(user, choosen_game)
    users.append(user)
    games.append(choosen_game)
    ratings.append(reward)
    rating_matrix[user, choosen_game] = reward
    '''
    print("user = {}, recommended_games = {}, choosen_game = {}".format(user,recommended_games,choosen_game))
    print("reward = {}\n".format(reward))
    '''
    
print("rating matrix: \n", str(rating_matrix))

rating matrix: 
 [[0. 4. 2. 0. 2. 3. 0. 4. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 4. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0. 0. 0. 3. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 2.]
 [3. 4. 2. 2. 0. 0. 0. 0. 0. 1.]
 [2. 0. 2. 1. 2. 2. 0. 3. 3. 1.]
 [0. 0. 1. 2. 1. 0. 3. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 2. 0. 2. 2. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0. 0. 0. 0. 2.]
 [2. 0. 1. 0. 1. 4. 3. 0. 3. 0.]
 [0. 3. 0. 0. 2. 0. 3. 3. 0. 0.]
 [3. 0. 0. 1. 0. 2. 0. 3. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 3. 0. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 4. 3. 1.]
 [0. 2. 2. 0. 2. 2. 0. 0. 2. 0.]
 [0. 3. 0. 0. 0. 0. 3. 0. 3. 0.]
 [2. 0. 0. 2. 0. 0. 2. 0. 0. 0.]
 [0. 4. 1. 2. 0. 0. 4. 4. 3. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 2. 0. 0. 2. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 4. 0. 0.]
 [1. 0. 2. 2. 0. 0. 0. 0. 1. 2.]
 [0. 3. 0. 2. 0. 0. 0. 3. 0. 0.]
 [2. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [3. 5. 0. 0. 2. 0. 4. 0. 

## Regression model

In [8]:
class RegressionModel(Model):
    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        self.flatten = Flatten()
        self.dot = Dot(axes=1)
        
    def call(self, inputs):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        y = self.dot([user_vecs, game_vecs])
        return y

## Deep regression model

In [9]:
class DeepRegressionModel(Model):

    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        
        self.flatten = Flatten()
        self.concat = Concatenate()
        
        self.dense1 = Dense(16, activation="relu")
        self.dense2 = Dense(8, activation="relu")
        
    def call(self, inputs, training=False):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        feature_inputs = inputs[2]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        # input_vecs = self.concat([user_vecs, game_vecs, self.flatten(feature_inputs)])
        input_vecs = self.concat([user_vecs, game_vecs])
        
        y = self.dense1(input_vecs)
        y = self.dense2(y)
        
        return y

## Embedding agent

In [10]:
class EmbeddingAgent:
    def __init__(self, X, Y, deepRegression=False):
        if deepRegression:
            self._model = DeepRegressionModel(64, nb_users, nb_games)
        else:
            self._model = RegressionModel(64, nb_users, nb_games)
        self._model.compile(optimizer="adam", loss='mae')
        self._model.fit(X, Y,
                  batch_size=64, epochs=100, validation_split=0.1,
                  shuffle=True)
        self._user_embeddings = self._model.get_weights()['user_embedding']
        self._game_embeddings = self._model.get_weights()['game_embedding']
    
    def act(self, user, available_games):
        user_embedding = self._user_embeddings[user]
        dot_products = self._game_embeddings @ user_embedding
        user_embedding_norm = np.linalg.norm(user_embedding)
        all_item_norms = np.linalg.norm(self._game_embeddings, axis=1)
        norm_products = user_embedding_norm * all_item_norms
        sims = dot_products / (norm_products)
        sims = np.argsort(sims)[::-1]
        mask = np.in1d(sims, available_games)
        sims = sims[mask]
        return sims[0]

In [12]:
deepRegression = True

users = np.array(users)
games = np.array(games)
ratings = np.array(ratings)

if deepRegression:
    features = []
    for i in range(len(users)):
        features.append(env.get_feature_vector(users[i], games[i]))
    features = np.float64(features)
    agent = EmbeddingAgent([users, games, features], ratings, deepRegression=deepRegression)
else:
    agent = EmbeddingAgent([users, games], ratings, deepRegression=deepRegression)

Train on 90 samples, validate on 10 samples
Epoch 1/100
90/90 [==============================] - 1s 8ms/sample - loss: 2.3339 - val_loss: 2.3706
Epoch 2/100
90/90 [==============================] - 0s 541us/sample - loss: 2.3216 - val_loss: 2.3600
Epoch 3/100
90/90 [==============================] - 0s 530us/sample - loss: 2.3082 - val_loss: 2.3479
Epoch 4/100
90/90 [==============================] - 0s 471us/sample - loss: 2.2938 - val_loss: 2.3343
Epoch 5/100
90/90 [==============================] - 0s 527us/sample - loss: 2.2783 - val_loss: 2.3198
Epoch 6/100
90/90 [==============================] - 0s 608us/sample - loss: 2.2615 - val_loss: 2.3041
Epoch 7/100
90/90 [==============================] - 0s 604us/sample - loss: 2.2434 - val_loss: 2.2869
Epoch 8/100
90/90 [==============================] - 0s 459us/sample - loss: 2.2237 - val_loss: 2.2677
Epoch 9/100
90/90 [==============================] - 0s 541us/sample - loss: 2.2021 - val_loss: 2.2462
Epoch 10/100
90/90 [===========

90/90 [==============================] - 0s 478us/sample - loss: 0.6359 - val_loss: 0.7605
Epoch 80/100
90/90 [==============================] - 0s 476us/sample - loss: 0.6346 - val_loss: 0.7590
Epoch 81/100
90/90 [==============================] - 0s 510us/sample - loss: 0.6339 - val_loss: 0.7580
Epoch 82/100
90/90 [==============================] - 0s 501us/sample - loss: 0.6343 - val_loss: 0.7585
Epoch 83/100
90/90 [==============================] - 0s 495us/sample - loss: 0.6331 - val_loss: 0.7616
Epoch 84/100
90/90 [==============================] - 0s 491us/sample - loss: 0.6328 - val_loss: 0.7638
Epoch 85/100
90/90 [==============================] - 0s 500us/sample - loss: 0.6326 - val_loss: 0.7655
Epoch 86/100
90/90 [==============================] - 0s 495us/sample - loss: 0.6318 - val_loss: 0.7677
Epoch 87/100
90/90 [==============================] - 0s 512us/sample - loss: 0.6309 - val_loss: 0.7702
Epoch 88/100
90/90 [==============================] - 0s 475us/sample - loss:

TypeError: list indices must be integers or slices, not str

In [12]:
# Running several trials
nb_iteration = 20 #how many trials
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(user, available_games)
    reward = env.update(user, choosen_game)
    rating_matrix[user, choosen_game] = reward
    print("user = {}, available games = {}, choosen_game = {}".format(user,available_games,choosen_game))
    print("reward = {}\n".format(reward))

user = 25, available games = [0 1 2 5 7 8 9], choosen_game = 2
reward = 1

user = 19, available games = [0 1 2 3 4 5 6 7 8 9], choosen_game = 7
reward = 2

user = 10, available games = [2 4 7 8], choosen_game = 2
reward = 1

user = 13, available games = [0 1 2 3 4 5 7 8 9], choosen_game = 9
reward = 3

user = 1, available games = [4 5 6 8], choosen_game = 8
reward = 2

user = 18, available games = [1 2 3 4 5 6 7 8 9], choosen_game = 2
reward = 3

user = 19, available games = [0 1 2 3 4 5 6 8 9], choosen_game = 8
reward = 2

user = 19, available games = [0 1 2 3 4 5 6 9], choosen_game = 4
reward = 1

user = 2, available games = [2 4 8 9], choosen_game = 9
reward = 2

user = 8, available games = [0 1 4 7 8 9], choosen_game = 9
reward = 2

user = 6, available games = [0 1 2 3 4 6 9], choosen_game = 9
reward = 4

user = 4, available games = [0 4 8 9], choosen_game = 9
reward = 2

user = 10, available games = [4 7 8], choosen_game = 8
reward = 1

user = 15, available games = [0 2 4 7 9], ch